# Intro to DLC2Action (mini)

<a href="https://colab.research.google.com/github/amathislab/DLC2action/blob/master/examples/minimal_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DLC2Action is a package for automatic behavior prediction. It offers implementation of SOTA models and keeps track of experiments.

To see how it works, we will experiment on a relatively small [publically available](https://github.com/ETHZ-INS/DLCAnalyzer/tree/master/data/OFT) dataset (Sturman, 2020). Run the code below to download the data.

This is a minimalistic version of this notebook, check out demo_notebook.ipynb for more information.

Note that the results we are getting here are not optimal because we are using very small numbers of epochs and trials to make the execution time fit within a short tutorial.

NOTE: If you want to run this in google colab, we recommend to use the notebook at [this link](https://colab.research.google.com/drive/1z7s7T4mf_z4WN7ag6XFNFtPCMLlWnuR0?usp=sharing).

## Setup - Installing Packages and Downloading Data

Downloading the data and installing the packages can take up to about 5-10 minutes.

First, let's download the data. </br>
*For Windows user*, you may need to install `wget` by downloading the .exe file [here](https://eternallybored.org/misc/wget/) (>1.10.0) and moving it to the System32 directory.

In [ ]:
!wget https://github.com/ETHZ-INS/DLCAnalyzer/archive/refs/heads/master.zip
!apt-get install unzip
!unzip master.zip
!mv DLCAnalyzer-master/data/OFT OFT_data

Now let's install `dlc2action`.

In [ ]:
!pip install dlc2action

## DLC2Action

First, we need to import the necessary packages and specify where our data is.

In [ ]:
import os
import pandas as pd
from dlc2action.project import Project

CURRENT_PATH = os.getcwd()
DATA_PATH = os.path.join(CURRENT_PATH, "OFT_data", "Output_DLC")
LABELS_PATH = os.path.join(CURRENT_PATH, "OFT_data", "Labels")
PROJECTS_PATH = os.path.join(CURRENT_PATH, "DLC2Action")

Then, we need to parse the data to make it compatible with `dlc2action` (one annotation file per video).

In [ ]:
annotations = f"{LABELS_PATH}/AllLabDataOFT_final.csv"

# loading the annotations
df_labels = pd.read_csv(annotations, sep=";")

# Getting the video name from the dlc filename
model = "DeepCut_resnet50_Blockcourse1May9shuffle1_1030000.csv"
df_labels["video"] = df_labels["DLCFile"].map(lambda dlc_file: dlc_file.split(model)[0])

# Splitting the data into one csv for each video
videos = df_labels["video"].unique()
for video in videos:
    df_video_labels = df_labels[df_labels["video"] == video]
    df_video_labels = df_video_labels.drop("video", axis=1)
    df_video_labels.to_csv(f"{LABELS_PATH}/{video}.csv")

Then we can move on to `DLC2Action`.

High-level methods in DLC2Action are almost exclusively accessed through the `dlc2action.project.Project` class. A project instance should loosely correspond to a specific goal (e.g. generating automatic annotations for dataset A with input format X). You can use it to optimize hyperparameters, run experiments, analyze results and generate new data. On the other hand, if you want to test different models, different model parameters, augmentations or types of extracted features it is better to work in the same project instance to compare these experiments.

**Best practices**
- When you need to do something with a different data type or unrelated files, it's better to create a new project to keep the experiment history easy to understand.
- Each project is associated with a folder on your computer that contains all settings, meta files and experiment outputs. Those folders are created in the folder at `projects_path`. It's generally a good idea to choose one and stick to it throughout projects.

### Creating a project

Let's begin!

We will create a project called `"oft"`, with `"dlc_track"` input and `"csv"` annotation format. 

You can run `Project.print_data_types()` and `Project.print_annotation_types()` to find out more about other options.

In [ ]:
# Project.remove_project("oft", projects_path=PROJECTS_PATH)
project = Project(
    "oft",
    data_path=DATA_PATH,
    annotation_path=LABELS_PATH,
    projects_path=PROJECTS_PATH,
    data_type="dlc_track",
    annotation_type="csv",
)

### Setting parameters

After the project is created, it's time to configure the parameter settings. 

The first step is to check which essential parameters are missing with `project.list_blanks()`.

In [ ]:
project.list_blanks()

We can copy this code, fill in the blanks and run it. 

We will also set the classes we want to ignore and number of epochs here. Normally the default should be fine but for the purpose of this tutorial we want to set it smaller so that our experiments can finish in time.

In [ ]:
project.update_parameters(
    {
        "data": {
            "data_suffix": "DeepCut_resnet50_Blockcourse1May9shuffle1_1030000.csv", # set; the data files should have the format of {video_id}{data_suffix}, e.g. video1_suffix.pickle, where video1 is the video is and _suffix.pickle is the suffix
            "canvas_shape": [928, 576], # list; the size of the canvas where the pose was defined
            "annotation_suffix": ".csv", # str | set, optional the suffix or the set of suffices such that the annotation files are named {video_id}{annotation_suffix}, e.g, video1_suffix.pickle where video1 is the video id and _suffix.pickle is the suffix
            "fps": 25, # int; fps (assuming the annotations are given in seconds, otherwise set any value)
            "ignored_classes": ["StartEnd", "_DEFAULT"]
        },
        "general": {
            "exclusive": True, # bool; if true, single-label classification is used; otherwise multi-label
        },
        "training": {
            "num_epochs": 15,
        }
    }
)

Now we're all set and can start training models.

### Hyperparameter search

There are many hyperparameters in model training, like the number of layers in a model or loss coefficients. The default settings for those parameters should generate reasonable results on most datasets but in order to get the most out of our data we can run a hyperparameter search. The default model is called C2F-TCN and is a temporal convolution neural network and can also be changed while updating the parameters.

The easiest way to find a good set of hyperparameters for your data is to run `project.run_default_hyperparameter_search()`.

In [ ]:
project.run_default_hyperparameter_search(
    "test_search",
    num_epochs=3,
    n_trials=10,
)

### Training models

Now we can train a model with the best hyperparameters. 

In [ ]:
project.run_episode(
    "test_best",
    load_search="test_search", # loading the search
    force=True, # when force=True, if an episode with this name already exists it will be overwritten -> use with caution!
)

### Evaluation

Now that we've trained our best models, we can analyze the results. In action segmentation tasks, the F1 score is given by the ratio of the product between precision and recall of a given class divided by their sum. Here we plot the evolution of F1 score during the model training. It can gives many indication on whether to stop the training or continue experimenting. 

In [ ]:
project.plot_episodes(
    ["test_best"],
    metrics=["f1"], # F1 score
    title="Best model training curve"
)

We can also check out more metrics now. See `project.help("metrics")` to see other options.

In [ ]:
project.evaluate(
    ["test_best"],
    parameters_update={
        "general": {"metric_functions": ["segmental_f1", "mAP", "f1"]},
        "metrics": {
            "f1": {"average": "none"}
        }
    }
)

### Using trained models

When you are happy with the results, you can use the model to generate predictions for new data.

Predictions here are given by the probabilities of each behavior being seen in each frame.

Let's generate a prediction using our trained model and look at one of the resulting files. Note that you can use multiple models and average over their predictions.

In [ ]:
project.run_prediction(
    "test_best_prediction",
    episode_names=[f"test_best"],
    force=True
)

In [ ]:
import pickle
import os


# picking a random file from the prediction folder
prediction_folder = project.prediction_path("test_best_prediction")
prediction_file = os.listdir(prediction_folder)[0]
prediction_file = os.path.join(prediction_folder, prediction_file)

with open(prediction_file, "rb") as f: # open the file
    prediction = pickle.load(f)

for key, value in prediction.items(): # explore the contents
    if key not in ["max_frames", "min_frames", "video_tag", "behaviors"]:
        print(f'{key}: {value.shape}')
    
behaviors_order = prediction["behaviors"]

start = 50
end = 70
action = "Unsupported"

index = behaviors_order.index(action)

print(f'The mean probability of {action} between frames {start} and {end} is {prediction["ind0"][index, start: end].mean()}')

We will now remove unnecessary data to clean the memory.

In [ ]:
project.remove_saved_features()
project.remove_extra_checkpoints()